In [1]:
import pandas as pd
import numpy as np

# Bokeh es la biblioteca que usaremos para las gráficas
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, DataTable, TableColumn, NumberFormatter
from bokeh.palettes import Dark2, YlGn9, Viridis, Pastel1

# output_notebook() activa el despliegue de gráficas en un notebook.
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv("answers-2021.csv", index_col=0)
# El salario de las personas de Mx se guarda en la columna salarymx y está en pesos.
# En la mayoría de los casos solo vamos a tomar en cuenta los datos de personas en Mx con salario en pesos.
df = df[(df['country'] == 'México')]
df.count()

created          1526
salarymx         1526
salaryusd        1526
extramx          1526
extrausd         1526
                 ... 
ben_vouchers      642
covid_remoto     1525
covid_salario    1525
covid_carga      1525
covid_apoyo      1032
Length: 161, dtype: int64

In [3]:
df.salarymx.describe()

count      1526.000000
mean      47248.365662
std       34417.260239
min           0.000000
25%       24000.000000
50%       40000.000000
75%       60000.000000
max      250000.000000
Name: salarymx, dtype: float64

In [4]:
df.experience.describe()

count    1526.000000
mean        9.966579
std         7.565638
min         0.000000
25%         4.000000
50%         8.000000
75%        14.000000
max        40.000000
Name: experience, dtype: float64

## Experiencia

In [5]:
df['exp_bin'] = pd.cut(df['experience'],bins=[-0.1,2,4,6,8,10,14,20,40])

exp = df.groupby('exp_bin')['salarymx'].agg(['count', 'median','mean', 'std'])
exp['std'] = round(exp['std']).astype(int)
# Groupby deja exp_bin como un índice, lo necesitamos como una columna normal así que damos reset_index.
exp = exp.reset_index()
exp.head(10)

,exp_bin,count,median,mean,std
0,"(-0.1, 2.0]",192,18000,19526.729167,12737
1,"(2.0, 4.0]",210,30000,32523.780952,18929
2,"(4.0, 6.0]",242,40000,46250.913223,31112
3,"(6.0, 8.0]",175,45000,50014.771429,29961
4,"(8.0, 10.0]",150,52500,59424.000000,41250
5,"(10.0, 14.0]",190,53000,57743.110526,33360
6,"(14.0, 20.0]",213,50000,56649.460094,35248
7,"(20.0, 40.0]",154,50000,62502.909091,43637


In [6]:

# Los valores en exp_bin son intervalos pero para la gráfica necesitamos que sean strings/categoricas.
exp_labels = ['0-2', '3-4', '5-6', '7-8', '9-10', '11-14', '15-20', '20+']
exp['exp_bin'] = exp_labels

# Tomo la paleta YlGn9 que tiene 9 colores y uso los primeros 8. No uso YlGn8 porque el último tono es muy claro.
# Pongo el resultado en una variable aux para luego poder invertir el orden de colores (más claro al principio)
aux = YlGn9[0:8]
exp['color'] = aux[::-1]

src = ColumnDataSource(exp)
p = figure(x_range=exp_labels, plot_height=400, plot_width=700)
p.vbar(source=src, x='exp_bin', top='median', width=0.95, color='color')
p.title.text = 'Salario medio de acuerdo a a la experiencia'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

hover = HoverTool()
hover.tooltips=[
    ('Experiencia', '@exp_bin años'),
    ('Observaciones', '@count'),
    ('Salario medio', '@median{$0,0}'),
]
p.add_tools(hover)

show(p)

## Comparación por género

In [7]:
# Agrupamos por experiencia y género. Llenamos con 0 los grupos sin valores.
gender = df.groupby(['exp_bin', 'gender'])['salarymx'].agg(['median','count']).fillna(0)
gender.head(25)

median  count
exp_bin      gender                
(-0.1, 2.0]  hombre  18000.0    118
             mujer   16500.0     68
             nb      17000.0      6
(2.0, 4.0]   hombre  33000.0    141
             mujer   24000.0     69
             nb          0.0      0
(4.0, 6.0]   hombre  42000.0    187
             mujer   29000.0     54
             nb      48000.0      1
(6.0, 8.0]   hombre  47000.0    137
             mujer   39000.0     37
             nb      80000.0      1
(8.0, 10.0]  hombre  57000.0    121
             mujer   39000.0     28
             nb      97000.0      1
(10.0, 14.0] hombre  55000.0    149
             mujer   50000.0     40
             nb      45000.0      1
(14.0, 20.0] hombre  55000.0    184
             mujer   40000.0     29
             nb          0.0      0
(20.0, 40.0] hombre  52300.0    128
             mujer   35000.0     25
             nb      50000.0      1

In [8]:
# El dataframe que arroja el groupby no se presta a graficar, así que lo reacomodamos en uno nuevo.
# El número de observaciones de género no binario es muy bajo y no arroja datos robustos así que lo omitiré.
data = {
    'exp': exp_labels,
    'hombre_salary' : list(gender.xs('hombre',level=1)['median']),
    'hombre_count' : list(gender.xs('hombre',level=1)['count']),
    'mujer_salary' : list(gender.xs('mujer',level=1)['median']),
    'mujer_count' : list(gender.xs('mujer',level=1)['count'])
}
genderdf = pd.DataFrame(data)
genderdf.head(10)

,exp,hombre_salary,hombre_count,mujer_salary,mujer_count
0,0-2,18000.0,118,16500.0,68
1,3-4,33000.0,141,24000.0,69
2,5-6,42000.0,187,29000.0,54
3,7-8,47000.0,137,39000.0,37
4,9-10,57000.0,121,39000.0,28
5,11-14,55000.0,149,50000.0,40
6,15-20,55000.0,184,40000.0,29
7,20+,52300.0,128,35000.0,25


In [9]:
# Teniendo esta estructura más amigable procedamos a generar lineas con sus tooltips.
src = ColumnDataSource(genderdf)
p = figure(x_range=exp_labels, plot_height=400)

renderer = p.line(x='exp',y='hombre_salary', source=src, color ='#1f77b4', line_width=2, legend_label='hombre')
p.add_tools(HoverTool(
            renderers=[renderer],
            tooltips=[
                ('Género', 'Hombre'),
                ('Experiencia', '@exp años'),
                ('Observaciones', '@hombre_count'),
                ('Salario', '@hombre_salary{$0,0}')
            ]
        ))

renderer = p.line(x='exp',y='mujer_salary', source=src, color ='#e617e6', line_width=2, legend_label='mujer')
p.add_tools(HoverTool(
            renderers=[renderer],
            tooltips=[
                ('Género', 'Mujer'),
                ('Experiencia', '@exp años'),
                ('Observaciones', '@mujer_count'),
                ('Salario', '@mujer_salary{$0,0}')
            ]
        ))


p.title.text = 'Comparativo por género'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')
p.legend.location = "bottom_right"

show(p)


In [10]:

    gender = df[(df['experience'] > 0)].groupby(["experience", "gender"])['salarymx'].agg(['median','count'])
    gender = gender.reset_index()
    gender = gender[(gender['count'] > 4)]
    gender['size'] = round(np.sqrt(gender['count']))*2

    # Agregamos una columna de color en base al valor de gender.
    gender['color'] = ['#1f77b4' if x =='hombre' else '#e617e6' for x in gender['gender']] 

    src = ColumnDataSource(gender)
    p = figure(plot_height = 400, plot_width = 600, sizing_mode="scale_both", toolbar_location=None)
    p.toolbar.active_drag = None
    p.toolbar.active_scroll = None
    p.circle(x='experience', y='median', source=src, size='size', color='color')
    p.title.text = 'Comparativo por género'
    p.xaxis.axis_label = 'Experiencia (años)'
    p.yaxis.axis_label = 'Salario bruto mensual (MXN)'
    p.yaxis.formatter = NumeralTickFormatter(format='$0 a')

    hover = HoverTool()
    hover.tooltips=[
        ('Género', '@gender'),    
        ('Experiencia', '@experience años'),
        ('Observaciones', '@count'),
        ('Salario medio', '@median{$0,0}'),
    ]

    p.add_tools(hover)
    show(p)

In [11]:
gx = pd.crosstab(index=df['city'], columns=df['gender'])
gx = gx[(gx['hombre'])>9]
gx

gender,hombre,mujer,nb
city,,,
Aguascalientes,18,0,1
CDMX,376,160,5
Chihuahua,11,3,0
Colima,12,6,0
Cuernavaca,14,1,0
Culiacán,13,2,0
Estado de México,31,11,0
Guadalajara,174,36,3
Hermosillo,33,7,0


## Inglés

In [12]:
ingles = df.groupby("english_label")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
ingles.head(20)

,count,median,mean,std
english_label,,,,
Nativo o bilingue (ILR 5),66,56500,67923.484848,47502.681074
Avanzado: Puedo conversar y escribir sin problemas sobre cualquier tema (ILR 4),381,56000,64356.351706,43043.974150
Profesional: Puedo interactuar profesionalmente con colegas y clientes (ILR 3),481,45100,49367.590437,28768.255477
Limitado: Me doy a entender pero con errores de gramática (ILR 2),441,30000,33231.308390,21155.121225
Elemental: Sé lo básico para sobrevivir (ILR 1),150,25000,30183.120000,23894.914275
Ninguno (ILR 0),7,22000,24285.714286,9604.066599


In [13]:
ingles = df.groupby(['english_num','exp_bin'])['salarymx'].agg(['median', 'count']).fillna(0)
data = {
    'exp': exp_labels,
    'Elemental': list(ingles.xs(1)['median']),
    'Limitado' : list(ingles.xs(2)['median']),
    'Profesional' : list(ingles.xs(3)['median']),
    'Avanzado' : list(ingles.xs(4)['median']),
    'Nativo' : list(ingles.xs(5)['median'])    
}

inglesdf = pd.DataFrame(data)

src = ColumnDataSource(inglesdf)

p = figure(x_range=exp_labels, plot_height=400, plot_width=600)

for col_name, color in zip(list(inglesdf.columns), Dark2[6]):
    if col_name == 'exp':
        continue

    p.add_tools(HoverTool(
        renderers= [p.line('exp', col_name, source=src, color=color, legend_label=col_name, line_width=2)],
        tooltips=[
            ('Ingles', col_name),
            ('Salario', '@'+col_name+'{$0,0}')
        ]
    ))

p.title.text = 'Salario de acuerdo al nivel de inglés'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Salario'
p.yaxis.formatter = NumeralTickFormatter(format='$0 a')
p.legend.location = "top_left"

    
show(p)


## Agrupación por ciudad y país

In [14]:
# Solo tomamos en cuenta a las personas con perfil de empleado (no freelancers, directivos ni emprendedores)
# Incluimos la experiencia como variable observada para obtener el promedio de experiencia por ciudad
cities = df[(df["profile"] == "godin")].groupby("city")['salarymx', 'experience'].agg(['count', 'median', 'mean', 'std'])
cities = cities.reset_index()

# El group by de múltiples columnas observadas con múltiples funciones agregadas nos genera que los nombre son tuplas
# así que hacemos este map para renombrar las columnas.
cities.columns = cities.columns.map('_'.join)

cities = cities[(cities["salarymx_count"]> 10)]
cities = cities[(cities["salarymx_median"]> 0)]
cities = cities.sort_values(by=['salarymx_median'], ascending=False)
src = ColumnDataSource(cities)
columns = [
        TableColumn(field="city_", title="Ciudad"),
        TableColumn(field="salarymx_count", title="n"),
        TableColumn(field="salarymx_median", title="Mediana", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="salarymx_mean", title="Media", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="salarymx_std", title="Des std", formatter=NumberFormatter(format='$0,0')),
        TableColumn(field="experience_mean", title="Experiencia promedio", formatter=NumberFormatter(format='0')),
    ]
table = DataTable(source=src,columns=columns,index_position=None, width=400)
show(table)

<ipython-input-14-995fe9ac340a>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cities = df[(df["profile"] == "godin")].groupby("city")['salarymx', 'experience'].agg(['count', 'median', 'mean', 'std'])


In [15]:
#guardamos en un csv para importar a sheets
cities.to_csv("tablas/cities.csv")

Hagamos el breakdown por país. Para ello nos basamos en la columna salaryusd.

In [16]:
# Volvemos a leer del csv para incluir datos de otros países
df2 = pd.read_csv("answers-2021.csv", index_col=0)
df2['salaryusd'] = np.where((df2.salaryusd == 0),round(df2.salarymx/19,0).astype(int), df2.salaryusd)
countries = df2.groupby("country")["salaryusd"].agg(['count', 'median', 'mean', 'std'])
countries = countries.reset_index()
countries = countries.sort_values(by=['median'], ascending=False)
countries.head(30)

,country,count,median,mean,std
6,Estados Unidos,18,11116.5,11411.111111,6093.012178
1,Canada,3,9000.0,10500.000000,3968.626967
4,Ecuador,5,2240.0,1902.000000,840.606924
5,España,2,2150.0,2150.000000,212.132034
8,México,1526,2105.0,2486.761468,1811.436312
2,Chile,19,2034.0,2355.578947,2093.359536
9,Otro,9,2000.0,5265.444444,7872.356336
7,Guatemala,2,1912.5,1912.500000,159.099026
10,Perú,13,1800.0,1913.923077,1080.276235
3,Colombia,14,1025.0,1274.071429,650.988061


## Lenguajes

El dilema con los lenguajes es que cada participante puede escoger varios lenguajes (máximo 3) que utiliza. Así que no podemos tener una única variable categórica para lenguaje, sino que tenemos una variable booleana (Y/N) por cada una de las opciones de lenguaje. Si tuvieramos una única variable, simplemente podríamos hacer un groupby y listo, pero al no tenerla, tenemos que "armar" nuestro dataframe.

Vamos a generar una lista donde cada elemento es a su vez una lista con la info de cada lenguaje (nombre del lenguaje, número de observaciones y salario medio). A partir de esa lista de listas generamos un dataframe y continuamos como de costumbre.

In [17]:
# Inicializamos nuestra lista maestra
lang_list = []
# Leemos la lista de lenguajes a partir de un catálogo.
import csv
with open("lang_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        lang_key = "lang_"+row['key']
        # Creamos una lista con el nombre del lenguaje, su num. de observaciones y salario medio, y agregamos dicha lista a nuestra lista maestra.
        lang_list.append([row['name'], df[(df[lang_key]=="Y")]["salarymx"].count(), df[(df[lang_key]=="Y")]["salarymx"].median(), df[(df[lang_key]=="Y")]["salarymx"].mean(), df[(df[lang_key]=="Y")]["salarymx"].std(), df[(df[lang_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
langdf = pd.DataFrame(lang_list, columns = ['lenguaje', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
langdf['popularidad'] = round(np.sqrt(langdf['n']/2)*2)
langdf.sort_values(by=['mediana'], ascending=False).head(40)





,lenguaje,n,mediana,media,std,exp,popularidad
20,Rust,4,73000.0,102988.750000,95819.329154,10.250000,3.0
6,Elixir,14,72500.0,71500.000000,27402.273909,7.785714,5.0
13,Objective C,4,66250.0,58225.000000,25042.680235,6.750000,3.0
19,Ruby,53,60000.0,71046.075472,40928.739867,7.867925,10.0
8,Go,40,55000.0,64162.500000,37395.390416,7.875000,9.0
14,Perl,2,53000.0,53000.000000,38183.766184,5.500000,2.0
21,Scala,16,52620.0,50121.125000,23552.075724,6.875000,6.0
0,Bash,76,45500.0,53171.157895,27307.489669,8.776316,12.0
12,Kotlin,32,45000.0,49453.125000,42223.934387,6.625000,8.0
17,Python,191,41500.0,48218.366492,34344.717547,8.240838,20.0


In [18]:
# Esto exporta el dataframe a un csv
langdf.to_csv("tablas/langs.csv")


In [19]:
from bokeh.models import LabelSet
from bokeh.models import Range1d

src = ColumnDataSource(langdf)
p = figure()
# Ponemos los circulos invisibles pero con tamaño para que sirvan los tooltips en hover.
p.circle(source=src, y='popularidad', x='mediana', line_color=None, fill_color=None, size=20)
p.yaxis.axis_label = 'Popularidad'

# La escala de popularidad tiene unidades arbitrarias así que prefiero evitar que se despliegue.
p.yaxis.major_label_text_font_size = '0pt'

p.xaxis.axis_label = 'Salario bruto mensual (MXN)'
p.xaxis.formatter = NumeralTickFormatter(format='$0 a')    
p.x_range = Range1d(10000, 70000)

labels = LabelSet(source=src, x='mediana', y='popularidad', text='lenguaje', level='glyph',
              x_offset=-10, y_offset=-5, render_mode='canvas', text_font_size="9pt", text_color='#1f77b4')
p.add_layout(labels)

hover = HoverTool()
hover.tooltips=[
    ('Lenguaje', '@lenguaje'),
    ('Observaciones', '@count'),
    ('Salario medio', '@salario{$0,0}'),
]

p.add_tools(hover)

show(p)

## Front end

In [20]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("front_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "front_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Tecnologia front', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Tecnologia front,n,mediana,media,std,exp
4,React,272,44000.0,51889.658088,39868.835180,7.007353
5,Unity,7,42000.0,36428.428571,22066.804419,10.285714
10,Qt,1,39000.0,39000.000000,NaN,7.000000
2,Ionic,28,38000.0,40443.357143,27143.078758,6.857143
9,Flutter,29,35000.0,34534.965517,22054.124931,7.344828
0,Angular,180,33350.0,39810.538889,29817.215310,7.694444
1,Cordova / PhoneGap,11,33000.0,40095.454545,25576.751695,7.545455
6,Vue,114,31500.0,40795.350877,33057.868827,6.403509
7,Xamarin,12,29500.0,42708.333333,45908.237691,7.500000
3,NativeScript,22,24950.0,31740.909091,16081.034647,9.545455


In [21]:
itemdf.to_csv("tablas/front.csv")

## Certificaciones

In [22]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("cert_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "cert_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Certificacion', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Certificacion,n,mediana,media,std,exp
9,Google Cloud Architect,14,68500.0,69664.285714,33778.136013,10.214286
11,Google Data Engineer,16,65000.0,64750.000000,21989.391382,9.000000
10,Google Mobile Web Specialist,3,60000.0,66000.000000,57236.352085,15.000000
2,AWS Solution Architect,39,57000.0,60207.435897,21183.553665,10.000000
3,Business Analyst (CCBA o CBAP),11,56000.0,66545.454545,22196.232277,20.181818
23,"Seguridad (Ethical Hacker, CISM, CISSP, CompTIA)",17,56000.0,62382.352941,58436.805988,14.705882
1,AWS Developer,40,56000.0,61823.550000,35050.112045,10.425000
19,PMP,59,55000.0,57675.254237,25736.333994,18.559322
6,"Enterprise Architect (SEI, IASA, Togaf, Zachman)",12,55000.0,74500.000000,58669.335339,18.750000
22,Six Sigma,41,55000.0,57224.146341,33297.418892,15.536585


In [23]:
itemdf.to_csv("tablas/certs.csv")

## Infraestructura

In [24]:
# Inicializamos nuestra lista maestra
item_list = []
# Leemos la lista de opciones a partir de un catálogo.
import csv
with open("infra_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        item_key = "infra_"+row['key']
        # Por cada elemento en la lista agregamos un renglon con el nombre de la categoria y estadisticas agregadas.
        item_list.append([row['name'], df[(df[item_key]=="Y")]["salarymx"].count(), df[(df[item_key]=="Y")]["salarymx"].median(), df[(df[item_key]=="Y")]["salarymx"].mean(), df[(df[item_key]=="Y")]["salarymx"].std(), df[(df[item_key]=="Y")]["experience"].mean()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
itemdf = pd.DataFrame(item_list, columns = ['Infra', 'n', 'mediana','media', 'std', 'exp'])
# Agregamos una columna de popularidad que se calcule en base a un valor amortiguado del número de observaciones.        
itemdf.sort_values(by=['mediana'], ascending=False).head(30)

,Infra,n,mediana,media,std,exp
5,Terraform,40,70500.0,69652.125000,28059.282912,8.800000
2,Kubernetes,37,70000.0,64972.972973,33806.168476,8.108108
1,Ansible,27,65000.0,66196.777778,31890.878421,8.666667
3,Chef,6,64450.0,62316.666667,23117.129291,12.000000
4,Puppet,5,60000.0,73622.600000,41980.223365,8.400000
0,Docker,83,55000.0,59755.614458,33203.437231,8.192771
6,OpenStack,4,52500.0,52750.000000,3201.562119,9.500000
7,OpenShift,6,46500.0,52633.333333,32259.055576,7.833333
8,VMware vCloud / vCenter,15,41000.0,39946.666667,16627.209484,13.466667


## Actividad

In [25]:
# Inicializamos nuestra lista maestra
master_list = []
# Leemos la lista de lenguajes a partir de un catálogo.
import csv
with open("activity_options.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        key = "act_"+row['key']
        master_list.append([row['name'], df[(df[key]=="Y")]["salarymx"].count(), df[(df[key]=="Y")]["salarymx"].median(), df[(df[key]=="Y")]["salarymx"].mean(), df[(df[key]=="Y")]["salarymx"].std()])

# Una vez que tenemos la lista maestra completa, creamos un dataframe indicando el nombre de las columnas.
act_df = pd.DataFrame(master_list, columns = ['actividad', 'n', 'mediana', 'mean', 'std'])
act_df.sort_values(by=['mediana'], ascending=False).head(30)

,actividad,n,mediana,mean,std
10,DirecciÃ³n / Estrategia,107,64000.0,71473.439252,42205.823219
17,Preventa / Tech sales,24,60000.0,70875.000000,45488.949924
2,Arquitectura y diseÃ±o de sistemas,396,55000.0,61482.171717,38916.502292
6,"GestiÃ³n de infraestructura (SysOps, DevOps)",124,50500.0,55724.645161,31477.826368
8,Coaching y mejora de procesos,104,50000.0,56376.548077,36772.411326
21,Venta y desarrollo de negocios,23,50000.0,66478.260870,52721.282200
16,Project management / CoordinaciÃ³n,228,48000.0,55390.587719,38169.530734
7,IngenierÃ­a de datos,72,44750.0,50895.833333,28683.181692
5,Ciencia de datos,54,43150.0,48506.444444,24386.976675
18,Seguridad de informaciÃ³n,54,43000.0,43111.555556,27284.270320


In [26]:
act_df.to_csv("tablas/actividad.csv")

## Estudios

In [27]:
educacion = df.groupby("education")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
educacion['mean'] = round(educacion['mean']).astype(int)
educacion

,count,median,mean,std
education,,,,
phd,20,61000,57695,29410.300290
maestria,269,48000,55836,34758.737034
posgrado,51,47000,53096,27721.141391
secundaria,2,46200,46200,33658.282784
prepa,32,40000,53220,42662.232787
universidad,771,40000,46195,34172.372941
pasante,347,35000,42134,34488.117521
tecnica,34,30000,34918,24315.716527


## ¿Dónde aprendiste a programar?

In [28]:
edutype = df.groupby("edutype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
edutype

,count,median,mean,std
edutype,,,,
autodidacta,239,46000,56806.096234,42971.229471
escuela,955,40500,46657.445026,32374.912686
trabajo,186,40000,47931.435484,34615.657465
online,99,33000,37301.282828,30383.496571
bootcamp,47,29000,28902.446809,12361.818534


Vemos que cursos online y bootcamps aparecen con los salarios más bajos. Pero eso puede ser engañoso, porque son opciones relativamente recientes y por lo tanto la gente que aprendió de esta manera no tiene tanta experiencia (que junto con el nivel de inglés es el factor que más influye en el salario). Así que ahora tomemos en cuenta solamente a los que tienen 5 años o menos de experiencia.

In [29]:
edutype = df[(df['experience']<=5)].groupby("edutype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
edutype

,count,median,mean,std
edutype,,,,
autodidacta,68,30556.5,35950.191176,28682.335837
bootcamp,34,28000.0,27873.970588,11555.422299
escuela,339,25000.0,29624.474926,21076.650314
trabajo,54,24050.0,30588.814815,23947.504719
online,56,23000.0,30746.142857,27783.402613


In [30]:
edutype.to_csv("tablas/edutype_5.csv")

Podemos ver que las cosas se emparejan mucho más. Incluso podemos decir que donde aprendiste a programar no es un factor significativo para tu salario. En otras palbras, lo que importa es lo que sabes, no como lo aprendiste.

Aprovechando que estamos en esto, vamos a ver si está cambiando la donde aprenden a programar las personas. Para ello, haremos un cruce de donde aprendieron vs años de experiencia.

In [31]:
exp_bin = df.groupby(['edutype','exp_bin'])['salarymx'].agg(['median', 'count']).fillna(0)
edutype_pct = exp_bin.groupby(level=1)['count'].apply(lambda x: x / float(x.sum()))
edutype_pct.head(20)

edutype      exp_bin     
autodidacta  (-0.1, 2.0]     0.088542
             (2.0, 4.0]      0.142857
             (4.0, 6.0]      0.148760
             (6.0, 8.0]      0.171429
             (8.0, 10.0]     0.186667
             (10.0, 14.0]    0.178947
             (14.0, 20.0]    0.173709
             (20.0, 40.0]    0.175325
bootcamp     (-0.1, 2.0]     0.088542
             (2.0, 4.0]      0.066667
             (4.0, 6.0]      0.016529
             (6.0, 8.0]      0.005714
             (8.0, 10.0]     0.020000
             (10.0, 14.0]    0.026316
             (14.0, 20.0]    0.009390
             (20.0, 40.0]    0.006494
escuela      (-0.1, 2.0]     0.619792
             (2.0, 4.0]      0.590476
             (4.0, 6.0]      0.619835
             (6.0, 8.0]      0.611429
Name: count, dtype: float64

In [32]:
data = {
    'exp': exp_labels,
    'escuela' : list(edutype_pct.xs('escuela')),
    'autodidacta': list(edutype_pct.xs('autodidacta')),
    'trabajo' : list(edutype_pct.xs('trabajo')),
    'online' : list(edutype_pct.xs('online')),
    'bootcamp' : list(edutype_pct.xs('bootcamp'))
}

edudf = pd.DataFrame(data)
edudf.sort_index(ascending=False, inplace=True)
edudf.head(20)

,exp,escuela,autodidacta,trabajo,online,bootcamp
7,20+,0.707792,0.175325,0.097403,0.012987,0.006494
6,15-20,0.633803,0.173709,0.154930,0.028169,0.009390
5,11-14,0.589474,0.178947,0.168421,0.036842,0.026316
4,9-10,0.660000,0.186667,0.106667,0.026667,0.020000
3,7-8,0.611429,0.171429,0.142857,0.068571,0.005714
2,5-6,0.619835,0.148760,0.111570,0.103306,0.016529
1,3-4,0.590476,0.142857,0.104762,0.095238,0.066667
0,0-2,0.619792,0.088542,0.083333,0.119792,0.088542


In [33]:


src = ColumnDataSource(edudf)
col_names = src.column_names

p = figure(x_range=exp_labels[::-1], plot_height=400, plot_width=600)

for col_name, color in zip(list(edudf.columns), Dark2[6]):
    if col_name == 'exp':
        continue

    p.add_tools(HoverTool(
        renderers= [p.line('exp', col_name, source=src, color=color, legend_label=col_name)],
        tooltips=[
            ('Formación', f'{col_name}'),
            ('Pct', '@'+col_name+'{0%}')
        ]
    ))

p.title.text = 'Evolución de donde aprendemos a programar'
p.xaxis.axis_label = 'Experiencia (años)'
p.yaxis.axis_label = 'Porcentaje'
p.yaxis.formatter = NumeralTickFormatter(format='0%')    
p.legend.location = (400,120)
    
show(p)


Llama la atención el crecimiento de los cursos online.

## Cliente local vs internacional

In [34]:
table = df.groupby("remote")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
table.head(20)

,count,median,mean,std
remote,,,,
Y,617,54000,60284.829822,37206.024464
N,909,33000,38399.632563,29265.612645


In [35]:
table = df.groupby("orgtype")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
table.head(20)

,count,median,mean,std
orgtype,,,,
startup,201,55000,64923.656716,48645.533738
itservices,712,43000,47795.644663,27934.322456
isv,173,40000,50101.034682,39926.306124
corp,286,35000,38598.475524,24639.388552
freelance,35,34000,59811.685714,63703.017036
gobierno,53,25000,30732.735849,26085.815307
uni,66,20350,24120.681818,16018.046551


## Covid-19

In [36]:
covid = df[["covid_carga","covid_remoto", "covid_salario", "covid_apoyo"]].copy()
covid.value_counts(["covid_remoto"])




covid_remoto
remote          1409
onsite           116
dtype: int64

## Special cases

In [37]:
segment = df[(df["city"] == "Guadalajara") & (df["experience"]>6)]
ingles = segment.groupby("english_label")["salarymx"].agg(['count', 'median', 'mean', 'std']).sort_values(by=['median'], ascending=False)
ingles.head(20)


,count,median,mean,std
english_label,,,,
Nativo o bilingue (ILR 5),6,125000,102666.666667,44862.753668
Avanzado: Puedo conversar y escribir sin problemas sobre cualquier tema (ILR 4),49,79000,80554.448980,33499.809251
Profesional: Puedo interactuar profesionalmente con colegas y clientes (ILR 3),47,60000,62978.765957,17708.077506
Limitado: Me doy a entender pero con errores de gramática (ILR 2),12,47500,51000.000000,26700.357505
Elemental: Sé lo básico para sobrevivir (ILR 1),2,16500,16500.000000,23334.523779


In [38]:
ingles.to_csv("tablas/gdl_ingles.csv")